In [1]:
%matplotlib inline

In [2]:
from __future__ import print_function, division

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
from sklearn.model_selection import KFold

!jupyter nbextension enable --py widgetsnbextension

plt.ion()

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [3]:
os.makedirs('/opt/ml/input/ensemble')
os.makedirs('/opt/ml/input/ensemble/pre-processing')
for i in range(18):
    os.makedirs('/opt/ml/input/ensemble/pre-processing/'+str(i))
for n in range(1,2):
    os.makedirs('/opt/ml/input/ensemble/data_'+str(n))
    os.makedirs('/opt/ml/input/ensemble/data_'+str(n)+'/train')
    os.makedirs('/opt/ml/input/ensemble/data_'+str(n)+'/val')
    for m in range(18):
        os.makedirs('/opt/ml/input/ensemble/data_'+str(n)+'/train/'+str(m))
        os.makedirs('/opt/ml/input/ensemble/data_'+str(n)+'/val/'+str(m))

In [4]:
def age(x):
    if x < 30:
        return 0
    elif x >= 58:
        return 2
    else:
        return 1

img_info = pd.read_csv('/opt/ml/input/data/train/train.csv')
img_info['age_label'] = img_info['age'].apply(lambda x : age(x))

for i in range(len(img_info)):
    if img_info.iloc[i,4] in ['004432_male_Asian_43','001498-1_male_Asian_23']:
        img_info.iloc[i,1] = 'female'
    elif img_info.iloc[i,4] in ['006359_female_Asian_18','006360_female_Asian_18','006361_female_Asian_18','006362_female_Asian_18','006363_female_Asian_18','006364_female_Asian_18']:
        img_info.iloc[i,1] = 'male'

In [5]:
for num in range(len(img_info)):
    for name in os.listdir('/opt/ml/input/data/train/images/'+img_info.iloc[num,4]):
        original_path = '/opt/ml/input/data/train/images/'+img_info.iloc[num,4]+'/'+name
        destination_path = '/opt/ml/input/ensemble/pre-processing/'
        if name[:4] == 'mask' and img_info.iloc[num,1] == 'male' and img_info.iloc[num,5] == 0:
            shutil.copyfile(original_path,destination_path+'0'+'/'+img_info.iloc[num,4]+name)
        elif name[:4] == 'mask' and img_info.iloc[num,1] == 'male' and img_info.iloc[num,5] == 1:
            shutil.copyfile(original_path,destination_path+'1'+'/'+img_info.iloc[num,4]+name)
        elif name[:4] == 'mask' and img_info.iloc[num,1] == 'male' and img_info.iloc[num,5] == 2:
            shutil.copyfile(original_path,destination_path+'2'+'/'+img_info.iloc[num,4]+name)
        elif name[:4] == 'mask' and img_info.iloc[num,1] == 'female' and img_info.iloc[num,5] == 0:
            shutil.copyfile(original_path,destination_path+'3'+'/'+img_info.iloc[num,4]+name)
        elif name[:4] == 'mask' and img_info.iloc[num,1] == 'female' and img_info.iloc[num,5] == 1:
            shutil.copyfile(original_path,destination_path+'4'+'/'+img_info.iloc[num,4]+name)
        elif name[:4] == 'mask' and img_info.iloc[num,1] == 'female' and img_info.iloc[num,5] == 2:
            shutil.copyfile(original_path,destination_path+'5'+'/'+img_info.iloc[num,4]+name)
        elif name[:4] == 'inco' and img_info.iloc[num,1] == 'male' and img_info.iloc[num,5] == 0:
            shutil.copyfile(original_path,destination_path+'6'+'/'+img_info.iloc[num,4]+name)
        elif name[:4] == 'inco' and img_info.iloc[num,1] == 'male' and img_info.iloc[num,5] == 1:
            shutil.copyfile(original_path,destination_path+'7'+'/'+img_info.iloc[num,4]+name)
        elif name[:4] == 'inco' and img_info.iloc[num,1] == 'male' and img_info.iloc[num,5] == 2:
            shutil.copyfile(original_path,destination_path+'8'+'/'+img_info.iloc[num,4]+name)
        elif name[:4] == 'inco' and img_info.iloc[num,1] == 'female' and img_info.iloc[num,5] == 0:
            shutil.copyfile(original_path,destination_path+'9'+'/'+img_info.iloc[num,4]+name)
        elif name[:4] == 'inco' and img_info.iloc[num,1] == 'female' and img_info.iloc[num,5] == 1:
            shutil.copyfile(original_path,destination_path+'10'+'/'+img_info.iloc[num,4]+name)
        elif name[:4] == 'inco' and img_info.iloc[num,1] == 'female' and img_info.iloc[num,5] == 2:
            shutil.copyfile(original_path,destination_path+'11'+'/'+img_info.iloc[num,4]+name)
        elif name[:4] == 'norm' and img_info.iloc[num,1] == 'male' and img_info.iloc[num,5] == 0:
            shutil.copyfile(original_path,destination_path+'12'+'/'+img_info.iloc[num,4]+name)
        elif name[:4] == 'norm' and img_info.iloc[num,1] == 'male' and img_info.iloc[num,5] == 1:
            shutil.copyfile(original_path,destination_path+'13'+'/'+img_info.iloc[num,4]+name)
        elif name[:4] == 'norm' and img_info.iloc[num,1] == 'male' and img_info.iloc[num,5] == 2:
            shutil.copyfile(original_path,destination_path+'14'+'/'+img_info.iloc[num,4]+name)
        elif name[:4] == 'norm' and img_info.iloc[num,1] == 'female' and img_info.iloc[num,5] == 0:
            shutil.copyfile(original_path,destination_path+'15'+'/'+img_info.iloc[num,4]+name)
        elif name[:4] == 'norm' and img_info.iloc[num,1] == 'female' and img_info.iloc[num,5] == 1:
            shutil.copyfile(original_path,destination_path+'16'+'/'+img_info.iloc[num,4]+name)
        elif name[:4] == 'norm' and img_info.iloc[num,1] == 'female' and img_info.iloc[num,5] == 2:
            shutil.copyfile(original_path,destination_path+'17'+'/'+img_info.iloc[num,4]+name)

In [6]:
kf = KFold(n_splits=10, shuffle=True, random_state= 42)

for num in range(18):
    tmp = np.array(os.listdir(destination_path+str(num)))
    cnt = 0
    for train_index, test_index in kf.split(tmp):
        cnt += 1
        if cnt == 1:
            train = tmp[train_index]
            test = tmp[test_index]
            for file_name in train:
                try:
                    shutil.copyfile('/opt/ml/input/ensemble/pre-processing/'+str(num)+'/'+file_name,
                                   '/opt/ml/input/ensemble/data_'+str(cnt)+'/train/'+str(num)+'/'+file_name)
                except:
                    pass
            for file_name in test:
                try:
                    shutil.copyfile('/opt/ml/input/ensemble/pre-processing/'+str(num)+'/'+file_name,
                                   '/opt/ml/input/ensemble/data_'+str(cnt)+'/val/'+str(num)+'/'+file_name)
                except:
                    pass

In [7]:
df = pd.read_csv('/opt/ml/input/data/eval/submission_tmp.csv')

for i in range(len(df)):
    shutil.copyfile('/opt/ml/input/data/eval/images/'+df.iloc[i,0],
                   '/opt/ml/input/ensemble/data_1/train/'+str(df.iloc[i,1])+'/'+df.iloc[i,0])